In [1]:
import os
import re
import json
import time
import librosa
import subprocess
import shlex
import torch
import fairseq
import numpy as np
import soundfile as sf
from tqdm import tqdm
if 'cd' not in globals():
    os.chdir('..')
    cd = True


In [2]:
def get_lang(d):
    s = d.split('_')[0]
    s = re.sub(r'\d+$', '', s.split('-')[0]) if re.search('[a-zA-Z]+', s) else s
    return s

In [9]:
def line_count(filename):
    return int(subprocess.check_output(['wc', '-l', filename]).split()[0])

In [ ]:
split = 'dev'
data_dir = 'data'
wav_scp = f'{data_dir}/{split}/wav.scp'
audio_folder = 'w2vaudio_wav'
feat_folder = 'w2vaudio_npy'
seen_utt = set()

nline = line_count(wav_scp)

with open(wav_scp, 'r') as f:
    for l in tqdm(f, total=nline):
        uttid, cmd = l.strip().strip('- |').split(' ', maxsplit=1)
        audio_path = f'{audio_folder}/{uttid}.wav'
        cmd = f' wav {audio_path} '.join(cmd.rsplit(' wav - ', maxsplit=1))
        
        if not os.path.isfile(audio_path):
#             assert os.path.isfile(f'{feat_folder}/{uttid}.npy')
#             continue
        
            process = subprocess.Popen(
                cmd,
                shell=True,
                stdout=subprocess.PIPE, 
                stderr=subprocess.PIPE
            )
            stdout, stderr = process.communicate()
            if stderr:
                if 'WARN' in str(stderr):
                    pass
                else:
                    assert False
        
        

In [30]:
segment_file

'data/dev/segment'

In [ ]:
data/dev/seg

In [36]:
split = 'train'
segment_file = f'{data_dir}/{split}/segments'
nline = line_count(segment_file)
sr = 16000
# seen_utt = set()
with open(segment_file, 'r') as f:
    for l in tqdm(f, total=nline):
        segid, audioid, start, end = l.split(' ')
        start, end = float(start), float(end)
        audio_path = f'{audio_folder}/{audioid}.wav'
        feat_path = f'{feat_folder}/{segid}.npy'
        if not os.path.isfile(feat_path):
            feats, curr_sample_rate = librosa.load(audio_path, sr=sr, offset=start, duration=end-start)
            assert curr_sample_rate == sr
            if len(feats.shape) == 2:
                feats = feats.mean(-1)
                assert len(feats) != 2
            np.save(feat_path, feats)

        assert segid not in seen_utt
        seen_utt.add(segid)

 60%|██████    | 239208/396806 [21:29<06:59, 375.53it/s] /home/hertin/Software/miniconda3/envs/espnet-lowres/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/hertin/Software/miniconda3/envs/espnet-lowres/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/hertin/Software/miniconda3/envs/espnet-lowres/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/hertin/Software/miniconda3/envs/espnet-lowres/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/hertin/Softwa

In [ ]:
# extract audio 

In [11]:
data_dir = 'data'

audio_folder = 'w2vaudio_wav'
feat_folder = 'w2vaudio_npy'

eval_folders = [f for f in os.listdir('data') if 'eval' in f]
seen_utt = set()
for eval_folder in eval_folders:
    wav_scp = f'{data_dir}/{eval_folder}/wav.scp'
    with open(wav_scp, 'r') as f:
        for l in tqdm(f, total=nline):
            uttid, cmd = l.strip().strip('- |').split(' ', maxsplit=1)
            audio_path = f'{audio_folder}/{uttid}.wav'
            cmd = f' wav {audio_path} '.join(cmd.rsplit(' wav - ', maxsplit=1))

            if not os.path.isfile(audio_path):
    #             assert os.path.isfile(f'{feat_folder}/{uttid}.npy')
    #             continue

                process = subprocess.Popen(
                    cmd,
                    shell=True,
                    stdout=subprocess.PIPE, 
                    stderr=subprocess.PIPE
                )
                stdout, stderr = process.communicate()
                if stderr:
                    if 'WARN' in str(stderr):
                        pass
                    else:
                        assert False


NameError: name 'split' is not defined

In [12]:
seen_utt = set()
for eval_folder in eval_folders:
    print(eval_folder)
    import time
    time.sleep(1)
    segment_file = f'{data_dir}/{eval_folder}/segments'
    nline = line_count(segment_file)
    sr = 16000
    # seen_utt = set()
    with open(segment_file, 'r') as f:
        for l in tqdm(f, total=nline):
            segid, audioid, start, end = l.split(' ')
            start, end = float(start), float(end)
            audio_path = f'{audio_folder}/{audioid}.wav'
            feat_path = f'{feat_folder}/{segid}.npy'
            if not os.path.isfile(feat_path):
                feats, curr_sample_rate = librosa.load(audio_path, sr=sr, offset=start, duration=end-start)
                assert curr_sample_rate == sr
                if len(feats.shape) == 2:
                    feats = feats.mean(-1)
                    assert len(feats) != 2
                np.save(feat_path, feats)

            assert segid not in seen_utt
            seen_utt.add(segid)

eval_Czech


100%|██████████| 1400/1400 [01:55<00:00, 12.14it/s]


eval_206


100%|██████████| 10494/10494 [08:54<00:00, 19.62it/s] 


eval_Spanish


...

100%|██████████| 9125/9125 [08:25<00:00, 18.05it/s] 


eval_Bulgarian


100%|██████████| 831/831 [01:00<00:00, 13.67it/s]


eval_107


100%|██████████| 9465/9465 [08:28<00:00, 18.62it/s] 


In [42]:
split = 'dev'
feat_folder = 'w2vaudio_npy'
with open(f'dump/{split}/deltafalse/data.json', 'r') as f:
    js = json.load(f)

In [ ]:
for uttid, v in tqdm(js['utts'].items(), total=len(js['utts'])):
    npy_file = f'{feat_folder}/{uttid}.npy'
    assert 'shape' in v['input'][0]
    assert 'feat' in v['input'][0]
    v['input'][0]['feat'] = npy_file
    v['input'][0]['filetype'] = 'npy'
    shape = np.load(npy_file, mmap_mode='r').shape[0]
    if type(shape) is int:
        shape =(shape, 1)
    elif len(shape) == 1:
        shape = (shape[0], 1)
    v['input'][0]['shape'] = shape


In [16]:
def convert_json_data(input_json_path):
    
    output_json_path = f'{input_json_path}.npy'
    if os.path.isfile(output_json_path):
        print(f'{output_json_path} already exists, skip')
        return
    with open(input_json_path, 'r') as fin:
        js = json.load(fin)["utts"]
        for uttid, v in tqdm(js.items(), total=len(js)):
            npy_file = f'{feat_folder}/{uttid}.npy'
            assert 'shape' in v['input'][0]
            assert 'feat' in v['input'][0]
            v['input'][0]['feat'] = npy_file
            v['input'][0]['filetype'] = 'npy'
            shape = np.load(npy_file, mmap_mode='r').shape[0]
            if type(shape) is int:
                shape =(shape, 1)
            elif len(shape) == 1:
                shape = (shape[0], 1)
            v['input'][0]['shape'] = shape
        print(f'saving modified npy json {output_json_path}')
        with open(output_json_path, 'w') as fout:
            json.dump({'utts': js}, fout, indent=2)
    

In [17]:
eval_folders = [f for f in os.listdir('data') if 'eval' in f]
feat_folder = 'w2vaudio_npy'
for eval_folder in eval_folders:
    input_json_path = f'dump/{eval_folder}/deltafalse/data.json'
    convert_json_data(input_json_path)
#     with open(f'dump/{eval_folder}/deltafalse/data.json', 'r') as fin:
#         js = json.load(fin)["utts"]
#         feat_folder = args.wav2vec_feat_folder
#         for uttid, v in js.items():
#             npy_file = f'{feat_folder}/{uttid}.npy'
#             assert 'shape' in v['input'][0]
#             assert 'feat' in v['input'][0]
#             v['input'][0]['feat'] = npy_file
#             v['input'][0]['filetype'] = 'npy'
#             shape = np.load(npy_file, mmap_mode='r').shape[0]
#             if type(shape) is int:
#                 shape =(shape, 1)
#             elif len(shape) == 1:
#                 shape = (shape[0], 1)
#             v['input'][0]['shape'] = shape
#         logging.warning(f'saving modified npy json {json_path}')
#         with open(f'dump/{eval_folder}/deltafalse/data.json.npy', 'w') as fout:
#             json.dump({'utts': js}, fout, indent=2)
#     json.dump(js, f, indent=2)
#     with open(f'dump/{eval_folder}/deltafalse/data.json.npy', 'w') as f:
#         json.dump(js, f, indent=2)

100%|██████████| 1400/1400 [00:01<00:00, 789.64it/s] 


saving modified npy json dump/eval_Czech/deltafalse/data.json.npy


100%|██████████| 10494/10494 [03:17<00:00, 53.23it/s] 


saving modified npy json dump/eval_206/deltafalse/data.json.npy


  1%|▏         | 11/820 [00:00<00:08, 99.75it/s]

...

saving modified npy json dump/eval_404/deltafalse/data.json.npy


100%|██████████| 831/831 [00:26<00:00, 31.88it/s]


saving modified npy json dump/eval_Bulgarian/deltafalse/data.json.npy


100%|██████████| 9465/9465 [03:38<00:00, 43.35it/s] 


saving modified npy json dump/eval_107/deltafalse/data.json.npy


In [ ]:

feat_folder = 'w2vaudio_npy'
with open(f'dump/{split}/deltafalse/data.json', 'r') as f:
    js = json.load(f)

In [11]:
cmd.split(' | sox -R -t wav - -t wav - rate 16000 dither')[0]

'/home/hertin/Workplace/SST/low-resource/espnet/tools/kaldi/tools/sph2pipe_v2.5/sph2pipe -f wav -p -c 1 /home/hertin/Workplace/SST/low-resource/espnet/egs/low-resource-language/asr1-cross/data/107/data/raw_train_data/audio/BABEL_BP_107_10033_20120208_180820_outLine.sph'

In [9]:
stderr

b'sox WARN rate: rate clipped 84 samples; decrease volume?\nsox WARN dither: dither clipped 71 samples; decrease volume?\n'

In [15]:
seen_utt = set()
for lang in all_langs:
    data_dir = lang2data[lang]
    nline = line_count(f'{data_dir}/wav.scp')
    
    with open(f'{data_dir}/wav.scp', 'r') as f:
        for l in tqdm(f, total=nline):
            # BG051_UTT001 shorten -x /home/hertin/Workplace/SST/dataset/babel/S0319/Bulgarian/adc/051/BG051_1.adc.shn - | sox -t raw -r 16000 -b 16 -e signed-integer - -t wav - |
            uttid, cmd = l.strip().strip('- |').split(' ', maxsplit=1)
            audio_path = f'{audio_folder}/{uttid}.wav'
            if os.isfile(audio_path):
                assert os.isfile(f'{feat_folder}/{uttid}.npy')
                continue

            feats, curr_sample_rate = sf.read(audio_path)
            if len(feats.shape) == 2:
                feats = feats.mean(-1)

            np.save(f'{feat_folder}/{uttid}.npy', feats)

            assert uttid not in seen_utt
            seen_utt.add(uttid)

100%|██████████| 724/724 [00:12<00:00, 57.96it/s]


In [ ]:
seen_utt = set()
data_dir = 'data'
audio_folder = 'w2vaudio_test_wav'
feat_folder = 'w2vaudio_npy'
for folder in os.listdir(data_dir):
    
    if 'eval' in folder:
        lang_dir = f'{data_dir}/{folder}'
        print(lang_dir)
        nline = line_count(f'{lang_dir}/wav.scp')
    
        with open(f'{lang_dir}/wav.scp', 'r') as f:
            for l in tqdm(f, total=nline):
                # BG051_UTT001 shorten -x /home/hertin/Workplace/SST/dataset/babel/S0319/Bulgarian/adc/051/BG051_1.adc.shn - | sox -t raw -r 16000 -b 16 -e signed-integer - -t wav - |
                uttid, cmd = l.strip().strip('- |').split(' ', maxsplit=1)
                cmd = cmd.split(' - | sox -R -t wav - -t wav')[0]
                audio_path = f'{audio_folder}/{uttid}.wav'
                cmd = f'{cmd} {audio_path}'

                process = subprocess.Popen(
                    cmd,
                    shell=True,
                    stdout=subprocess.PIPE, 
                    stderr=subprocess.PIPE
                )
                stdout, stderr = process.communicate()
                if stderr:
                    assert False
#                 print(stdout, stderr)
                
                feats, curr_sample_rate = sf.read(audio_path)
                if len(feats.shape) == 2:
                    feats = feats.mean(-1)

                np.save(f'{feat_folder}/{uttid}.npy', feats)

                assert uttid not in seen_utt
                seen_utt.add(uttid)


  0%|          | 0/1400 [00:00<?, ?it/s]

data/eval_Czech


  0%|          | 2/820 [00:00<01:03, 12.85it/s]

data/eval_French


  0%|          | 0/446 [00:00<?, ?it/s]

...

  0%|          | 3/865 [00:00<00:30, 27.91it/s]

data/eval_Polish


  0%|          | 2/763 [00:00<00:43, 17.61it/s]

data/eval_German


 29%|██▉       | 225/763 [00:09<00:46, 11.51it/s]

In [18]:
cmd

'shorten -x /home/hertin/Workplace/SST/dataset/babel/S0319/Bulgarian/adc/040/BG040_1.adc.shn - | sox -t raw -r 16000 -b 16 -e signed-integer - -t wav - w2vaudio_test_wav/BG040_UTT001.wav'

In [ ]:
sox -t raw -r 16000 -b 16 -e signed-integer - -t wav - | sox -R -t wav - -t wav - rate 16000 dither

In [ ]:
for folder in os.listdir(data_dir):
    
    if 'eval' in folder:
        lang_dir = f'{data_dir}/{folder}'
        print(lang_dir)
        nline = line_count(f'{lang_dir}/wav.scp')
    
        with open(f'{lang_dir}/wav.scp', 'r') as f:
            for l in tqdm(f, total=nline):
                # BG051_UTT001 shorten -x /home/hertin/Workplace/SST/dataset/babel/S0319/Bulgarian/adc/051/BG051_1.adc.shn - | sox -t raw -r 16000 -b 16 -e signed-integer - -t wav - |
                uttid, cmd = l.strip().strip('- |').split(' ', maxsplit=1)
                audio_path = f'{audio_folder}/{uttid}.wav'
                feats, curr_sample_rate = sf.read(audio_path)
                if len(feats.shape) == 2:
                    feats = feats.mean(-1)

                np.save(f'{feat_folder}/{uttid}.npy', feats)

                assert uttid not in seen_utt
                seen_utt.add(uttid)

In [49]:
f'{feat_folder}/{uttid}.npy'

'w2vfeat_npy/CZ009_UTT140.npy'

In [52]:
np.load(f'{feat_folder}/{uttid}.npy', mmap_mode='r').shape[1:]

(512, 649)

In [58]:
z.squeeze(0).T.shape

torch.Size([649, 512])

In [56]:
z.squeeze(0).shape


torch.Size([512, 649])

In [23]:
cp_path = '/home/hertin/Workplace/SST/low-resource/fairseq/examples/wav2vec/scripts/xlsr_53_56k.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()
device = torch.device('cuda:1')
model = model.to(device)
# wav_input_16khz = torch.randn(2,10000)
start = time.time()
wav_input = torch.from_numpy(audio).to(device).unsqueeze(0)

z = model.feature_extractor(wav_input)
feat = z.detach().cpu().numpy()
end = time.time()
print(f'cost {end-start} s')
# c = model.feature_aggregator(z)

cost 0.011581659317016602 s


In [24]:
cp_path = '/home/hertin/Workplace/SST/low-resource/fairseq/examples/wav2vec/scripts/xlsr_53_56k.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()

# wav_input_16khz = torch.randn(2,10000)
start = time.time()
wav_input = torch.from_numpy(audio).unsqueeze(0)

z = model.feature_extractor(wav_input)
end = time.time()
print(f'cost {end-start} s')
# c = model.feature_aggregator(z)

cost 0.9544930458068848 s


In [48]:
z.size()

torch.Size([1, 512, 649])